In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 13.5MB/s 
     |████████████████████████████████| 1.1MB 60.2MB/s 
     |████████████████████████████████| 890kB 63.5MB/s 
     |████████████████████████████████| 2.9MB 49.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e746800b9f006c12784e7c985d33894e53d5f6ca15525a4c6ff1af1a9684c770
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
 !pip install emoji

     |████████████████████████████████| 51kB 4.7MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=68d5eeb1bda8e47e0d6f95dfca6fe6d061a3e482efad8a5a386c29b1de7a1aa0
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [ ]:
import warnings

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", DeprecationWarning)

import tensorflow as tf
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import pandas as pd
import io
import os
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from statistics import mode

import re
import emoji
import random

import nltk
nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, TweetTokenizer
from nltk.corpus import wordnet, stopwords
from imblearn.over_sampling import SMOTE

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'Tesla V100-SXM2-16GB'

In [ ]:
def preprocess(df):
    
    #removes URL
    pattern = r'https.?://[^\s]+[\s]?'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes usernames/mentions
    pattern = r'@[^\s]+'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes emoji and smiley
    pattern = re.compile("["
                         u"\U0001F600-\U0001F64F"
                         u"\U0001F300-\U0001F5FF"
                         u"\U0001F680-\U0001F6FF"
                         u"\U0001F1E0-\U0001F1FF"
                         u"\U00002500-\U00002BEF"
                         u"\U00002702-\U000027B0"
                         u"\U00002702-\U000027B0"
                         u"\U000024C2-\U0001F251"
                         u"\U0001f926-\U0001f937"
                         u"\U00010000-\U0010ffff"
                         u"\u2640-\u2642"
                         u"\u2600-\u2B55"
                         u"\u200d"
                         u"\u23cf"
                         u"\u23e9"
                         u"\u231a"
                         u"\ufe0f"
                         u"\u3030"
                         "]+", flags=re.UNICODE)
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes numbers
    pattern = r'\d+'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes punctuation
    pattern = r"[^\w\s]"
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)

    #removes stop words
    stop_words = stopwords.words("english")    
    remove_stop_words = lambda row: " ".join([token for token in row.split(" ")
                                              if token not in stop_words])
    df["tweet"] = df["tweet"].apply(remove_stop_words)
    
    #removes extra spaces
    pattern = r"[\s]+"
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)
    
    return(df)

In [ ]:
def train_validate_test_split(df,seed, train_percent=.8, validate_percent=.125):
    train, test = train_test_split(df, train_size=train_percent, stratify=df['label'])
    train, validate = train_test_split(train, test_size=validate_percent, stratify=train['label'])
    return train, validate, test

def tokenize_data(df, sampling=False):
    sentences = ["[CLS] " + query + " [SEP]" for query in df['tweet']]
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    
    MAX_LEN = 128
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    
    if(sampling==True):
      input_ids, labels = oversampling(input_ids, df.label)
    else:
      labels = df['label'].copy()

    # Create attention masks
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return input_ids, attention_masks, labels

def Data_Loader(inputs_ids, attention_masks, labels, batch_size=16): 
    data = TensorDataset(torch.LongTensor(inputs_ids), torch.LongTensor(attention_masks), torch.LongTensor(labels.ravel()))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    return dataloader

def oversampling(ids, label):
  over = SMOTE(kind='svm', k_neighbors=2)
  sampled = over.fit_sample(ids, label)
  labels = sampled[1].copy()
  input_ids = sampled[0].copy()
  return input_ids, labels

In [ ]:
def model_train(model, train_dataloader, validation_dataloader):
    epochs = 5
    for _ in trange(epochs, desc="Epoch"):  
        model.train()
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            optimizer.zero_grad()
            loss, _ = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss.backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1
        print("Train loss: {}".format(tr_loss/nb_tr_steps))

        ## VALIDATION

        model.eval()
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        for batch in validation_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            with torch.no_grad():
                logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0]
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
            eval_accuracy += tmp_eval_accuracy
            nb_eval_steps += 1
        validation_accuracy = (eval_accuracy/nb_eval_steps)
        print("Validation Accuracy: {}".format(validation_accuracy))
    return validation_accuracy

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def model_test(model,prediction_dataloader):
    model.eval()
    predictions , true_labels = [], []
    for batch in prediction_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions+=list(np.argmax(logits, axis=1).flatten())
        true_labels+=list(label_ids.flatten())
    weighted_f1_score = f1_score(true_labels, predictions, average= 'weighted')
    print("Weighted F1 Score:",weighted_f1_score)
    macro_f1_score = f1_score(true_labels, predictions, average= 'macro')
    print("Macro F1 Score:",macro_f1_score)    
    test_accuracy_score = accuracy_score(true_labels, predictions)
    print("Accuracy score:", test_accuracy_score, "\n")
    print("="*100)
    return weighted_f1_score, macro_f1_score

In [ ]:
def model_initialise(dataloader_len):
    model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels = 2, output_attentions = False, output_hidden_states = False).cuda()
    optimizer = optim.AdamW(params = model.parameters(), lr=1e-5)
    epochs = 5
    total_steps = dataloader_len * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
    return model, optimizer, scheduler

In [ ]:
print_stmts= []
languages = ['French','Arabic','English']
directory = './'
for lang in languages:
    df = pd.read_csv(os.path.join(directory, lang+'.csv'))
    df = preprocess(df)
    weighted = []
    macro = []
    seeds = [2018, 2019, 2020, 2021, 2022]
    for seed in seeds:
        np.random.seed(seed)
        train_df, validation_df, test_df = train_validate_test_split(df, seed)
        train_input_ids, train_attention_masks, train_labels = tokenize_data(train_df)
        train_dataloader = Data_Loader(train_input_ids, train_attention_masks, train_labels)
        validation_input_ids, validation_attention_masks, validation_labels = tokenize_data(validation_df)
        validation_dataloader = Data_Loader(validation_input_ids, validation_attention_masks, validation_labels)
        print("\nModel Summary:")
        print('Language:', lang)
        print('Seed value:', seed)
        model, optimizer, scheduler = model_initialise(len(train_dataloader))
        validation_accuracy = model_train(model, train_dataloader, validation_dataloader)
        test_input_ids, test_attention_masks, test_labels = tokenize_data(test_df)
        test_dataloader = Data_Loader(test_input_ids, test_attention_masks, test_labels)
        w, m = model_test(model, test_dataloader)
        weighted.append(w)
        macro.append(m)
    print("The Average  Weighted F1-Score of the Language ", lang, "is:",sum(weighted)/ len(weighted))
    print("The Average  Macro F1-Score of the Language ", lang, "is:",sum(macro)/ len(macro))
    print("="*200)
    print_stmts.append("Average Weighted F1-Score "+str(sum(weighted)/len(weighted))+" and Average Macro F1-Score "+str(sum(macro)/len(macro))+"of "+ str(lang))
    for i in print_stmts:
        print(i,"\n")


Model Summary:
Language: French
Seed value: 2018


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

Train loss: 0.6648084995923219


Epoch:  20%|██        | 1/5 [00:08<00:32,  8.12s/it]

Validation Accuracy: 0.6640625
Train loss: 0.6358429474963082


...

Epoch:  80%|████████  | 4/5 [46:33<11:38, 698.62s/it]

Validation Accuracy: 0.8645422357106728
Train loss: 0.24019365908233611


Epoch: 100%|██████████| 5/5 [58:13<00:00, 698.70s/it]

Validation Accuracy: 0.8651239251390996


Weighted F1 Score: 0.8609575842541556
Macro F1 Score: 0.7716388924058272
Accuracy score: 0.8643175566726738 

The Average  Weighted F1-Score of the Language  English is: 0.8611439578728886
The Average  Macro F1-Score of the Language  English is: 0.7738952465426893
Average Weighted F1-Score0.5617914925533054and Average Macro F1-Score0.4359955364259525of the languageFrench 

Average Weighted F1-Score0.8988776449510304and Average Macro F1-Score0.8471303948622133of the languageArabic 

Average Weighted F1-Score0.8611439578728886and Average Macro F1-Score0.7738952465426893of the languageEnglish 

====================================================================================================English====================================================================================================


In [ ]:
# Model Summary:
# Language: French
# Seed value: 2018
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.6648084995923219
# Epoch:  20%|██        | 1/5 [00:08<00:32,  8.12s/it]Validation Accuracy: 0.6640625
# Train loss: 0.6358429474963082
# Epoch:  40%|████      | 2/5 [00:16<00:24,  8.11s/it]Validation Accuracy: 0.6734375
# Train loss: 0.6332687608621739
# Epoch:  60%|██████    | 3/5 [00:24<00:16,  8.11s/it]Validation Accuracy: 0.66875
# Train loss: 0.6331957964985458
# Epoch:  80%|████████  | 4/5 [00:32<00:08,  8.10s/it]Validation Accuracy: 0.6734375
# Train loss: 0.6321470649154098
# Epoch: 100%|██████████| 5/5 [00:40<00:00,  8.10s/it]Validation Accuracy: 0.6734375

# Weighted F1 Score: 0.5403407264545163
# Macro F1 Score: 0.40196078431372545
# Accuracy score: 0.6721311475409836 

# ====================================================================================================

# Model Summary:
# Language: French
# Seed value: 2019
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.662671575391734
# Epoch:  20%|██        | 1/5 [00:08<00:32,  8.12s/it]Validation Accuracy: 0.678125
# Train loss: 0.64702868020093
# Epoch:  40%|████      | 2/5 [00:16<00:24,  8.12s/it]Validation Accuracy: 0.6640625
# Train loss: 0.6367832019373223
# Epoch:  60%|██████    | 3/5 [00:24<00:16,  8.11s/it]Validation Accuracy: 0.678125
# Train loss: 0.6208177716643722
# Epoch:  80%|████████  | 4/5 [00:32<00:08,  8.11s/it]Validation Accuracy: 0.678125
# Train loss: 0.6147772862955376
# Epoch: 100%|██████████| 5/5 [00:40<00:00,  8.10s/it]Validation Accuracy: 0.678125

# Weighted F1 Score: 0.5403407264545163
# Macro F1 Score: 0.40196078431372545
# Accuracy score: 0.6721311475409836 

# ====================================================================================================

# Model Summary:
# Language: French
# Seed value: 2020
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.6607458823257022
# Epoch:  20%|██        | 1/5 [00:08<00:32,  8.12s/it]Validation Accuracy: 0.6640625
# Train loss: 0.6521421461193649
# Epoch:  40%|████      | 2/5 [00:16<00:24,  8.11s/it]Validation Accuracy: 0.6828125
# Train loss: 0.6325672495144384
# Epoch:  60%|██████    | 3/5 [00:24<00:16,  8.11s/it]Validation Accuracy: 0.6546875
# Train loss: 0.6317089762952592
# Epoch:  80%|████████  | 4/5 [00:32<00:08,  8.10s/it]Validation Accuracy: 0.66875
# Train loss: 0.636870437198215
# Epoch: 100%|██████████| 5/5 [00:40<00:00,  8.10s/it]Validation Accuracy: 0.66875

# Weighted F1 Score: 0.5403407264545163
# Macro F1 Score: 0.40196078431372545
# Accuracy score: 0.6721311475409836 

# ====================================================================================================

# Model Summary:
# Language: French
# Seed value: 2021
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.6446397966808743
# Epoch:  20%|██        | 1/5 [00:08<00:32,  8.12s/it]Validation Accuracy: 0.6734375
# Train loss: 0.6372946743611936
# Epoch:  40%|████      | 2/5 [00:16<00:24,  8.11s/it]Validation Accuracy: 0.66875
# Train loss: 0.6318548261015503
# Epoch:  60%|██████    | 3/5 [00:24<00:16,  8.11s/it]Validation Accuracy: 0.6484375
# Train loss: 0.5878503697889822
# Epoch:  80%|████████  | 4/5 [00:32<00:08,  8.10s/it]Validation Accuracy: 0.671875
# Train loss: 0.5733188401769709
# Epoch: 100%|██████████| 5/5 [00:40<00:00,  8.10s/it]Validation Accuracy: 0.659375

# Weighted F1 Score: 0.6475945569484625
# Macro F1 Score: 0.5721345448748605
# Accuracy score: 0.6844262295081968 

# ====================================================================================================

# Model Summary:
# Language: French
# Seed value: 2022
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.647764918980775
# Epoch:  20%|██        | 1/5 [00:08<00:32,  8.13s/it]Validation Accuracy: 0.6828125
# Train loss: 0.6489278022889737
# Epoch:  40%|████      | 2/5 [00:16<00:24,  8.12s/it]Validation Accuracy: 0.6640625
# Train loss: 0.6414708925618066
# Epoch:  60%|██████    | 3/5 [00:24<00:16,  8.12s/it]Validation Accuracy: 0.6734375
# Train loss: 0.6140303335807942
# Epoch:  80%|████████  | 4/5 [00:32<00:08,  8.12s/it]Validation Accuracy: 0.6734375
# Train loss: 0.5890485224900422
# Epoch: 100%|██████████| 5/5 [00:40<00:00,  8.11s/it]Validation Accuracy: 0.6953125

# Weighted F1 Score: 0.5403407264545163
# Macro F1 Score: 0.40196078431372545
# Accuracy score: 0.6721311475409836 

# ====================================================================================================
# The Average  Weighted F1-Score of the Language  French is: 0.5617914925533054
# The Average  Macro F1-Score of the Language  French is: 0.4359955364259525
# ========================================================================================================================================================================================================
# Average Weighted F1-Score0.5617914925533054and Average Macro F1-Score0.4359955364259525of the languageFrench 

# ====================================================================================================French====================================================================================================

# Model Summary:
# Language: Arabic
# Seed value: 2018
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.49767999532889196
# Epoch:  20%|██        | 1/5 [00:38<02:33, 38.35s/it]Validation Accuracy: 0.8513513513513513
# Train loss: 0.35226357879307796
# Epoch:  40%|████      | 2/5 [01:16<01:55, 38.34s/it]Validation Accuracy: 0.8682432432432432
# Train loss: 0.31190237089052913
# Epoch:  60%|██████    | 3/5 [01:55<01:16, 38.35s/it]Validation Accuracy: 0.857545045045045
# Train loss: 0.28807886981764647
# Epoch:  80%|████████  | 4/5 [02:33<00:38, 38.36s/it]Validation Accuracy: 0.8643018018018018
# Train loss: 0.2649318701990946
# Epoch: 100%|██████████| 5/5 [03:11<00:00, 38.37s/it]Validation Accuracy: 0.8626126126126127

# Weighted F1 Score: 0.8838678180771445
# Macro F1 Score: 0.8193414965986395
# Accuracy score: 0.8886010362694301 

# ====================================================================================================

# Model Summary:
# Language: Arabic
# Seed value: 2019
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.4823182058029287
# Epoch:  20%|██        | 1/5 [00:38<02:33, 38.39s/it]Validation Accuracy: 0.856418918918919
# Train loss: 0.325013680735559
# Epoch:  40%|████      | 2/5 [01:16<01:55, 38.40s/it]Validation Accuracy: 0.8868243243243243
# Train loss: 0.2636119943537463
# Epoch:  60%|██████    | 3/5 [01:55<01:16, 38.41s/it]Validation Accuracy: 0.902027027027027
# Train loss: 0.22077986609014705
# Epoch:  80%|████████  | 4/5 [02:33<00:38, 38.43s/it]Validation Accuracy: 0.8868243243243243
# Train loss: 0.18858723704681152
# Epoch: 100%|██████████| 5/5 [03:12<00:00, 38.43s/it]Validation Accuracy: 0.8896396396396395

# Weighted F1 Score: 0.8943275265286397
# Macro F1 Score: 0.8411522633744857
# Accuracy score: 0.8946459412780656 

# ====================================================================================================

# Model Summary:
# Language: Arabic
# Seed value: 2020
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.5269669220438153
# Epoch:  20%|██        | 1/5 [00:38<02:33, 38.39s/it]Validation Accuracy: 0.7865990990990991
# Train loss: 0.417481966404699
# Epoch:  40%|████      | 2/5 [01:16<01:55, 38.42s/it]Validation Accuracy: 0.875
# Train loss: 0.31447276191448603
# Epoch:  60%|██████    | 3/5 [01:55<01:16, 38.43s/it]Validation Accuracy: 0.8817567567567568
# Train loss: 0.26241794375218747
# Epoch:  80%|████████  | 4/5 [02:33<00:38, 38.43s/it]Validation Accuracy: 0.8834459459459459
# Train loss: 0.23469854246340985
# Epoch: 100%|██████████| 5/5 [03:12<00:00, 38.44s/it]Validation Accuracy: 0.8783783783783784

# Weighted F1 Score: 0.9161721625618257
# Macro F1 Score: 0.8742717452258387
# Accuracy score: 0.9162348877374784 

# ====================================================================================================

# Model Summary:
# Language: Arabic
# Seed value: 2021
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.4744490548146991
# Epoch:  20%|██        | 1/5 [00:38<02:33, 38.44s/it]Validation Accuracy: 0.8733108108108109
# Train loss: 0.3323861544467802
# Epoch:  40%|████      | 2/5 [01:16<01:55, 38.44s/it]Validation Accuracy: 0.8834459459459459
# Train loss: 0.2704096885295365
# Epoch:  60%|██████    | 3/5 [01:55<01:16, 38.44s/it]Validation Accuracy: 0.8778153153153152
# Train loss: 0.23662704582084118
# Epoch:  80%|████████  | 4/5 [02:33<00:38, 38.44s/it]Validation Accuracy: 0.8811936936936936
# Train loss: 0.210832382383661
# Epoch: 100%|██████████| 5/5 [03:12<00:00, 38.44s/it]Validation Accuracy: 0.8834459459459459

# Weighted F1 Score: 0.9017544599136803
# Macro F1 Score: 0.8543881997255496
# Accuracy score: 0.9006908462867013 

# ====================================================================================================

# Model Summary:
# Language: Arabic
# Seed value: 2022
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.4759269176036354
# Epoch:  20%|██        | 1/5 [00:38<02:33, 38.43s/it]Validation Accuracy: 0.8676801801801801
# Train loss: 0.32127251455516326
# Epoch:  40%|████      | 2/5 [01:16<01:55, 38.43s/it]Validation Accuracy: 0.8868243243243243
# Train loss: 0.2551351697294144
# Epoch:  60%|██████    | 3/5 [01:55<01:16, 38.43s/it]Validation Accuracy: 0.893581081081081
# Train loss: 0.21160431389557563
# Epoch:  80%|████████  | 4/5 [02:33<00:38, 38.44s/it]Validation Accuracy: 0.8952702702702703
# Train loss: 0.1770935259341431
# Epoch: 100%|██████████| 5/5 [03:12<00:00, 38.44s/it]Validation Accuracy: 0.8918918918918919

# Weighted F1 Score: 0.8982662576738617
# Macro F1 Score: 0.8464982693865528
# Accuracy score: 0.8989637305699482 

# ====================================================================================================
# The Average  Weighted F1-Score of the Language  Arabic is: 0.8988776449510304
# The Average  Macro F1-Score of the Language  Arabic is: 0.8471303948622133
# ========================================================================================================================================================================================================
# Average Weighted F1-Score0.5617914925533054and Average Macro F1-Score0.4359955364259525of the languageFrench 

# Average Weighted F1-Score0.8988776449510304and Average Macro F1-Score0.8471303948622133of the languageArabic 

# ====================================================================================================Arabic====================================================================================================

# Model Summary:
# Language: English
# Seed value: 2018
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.3683812754354287
# Epoch:  20%|██        | 1/5 [11:38<46:33, 698.49s/it]Validation Accuracy: 0.859129994941831
# Train loss: 0.3160639904215754
# Epoch:  40%|████      | 2/5 [23:17<34:56, 698.69s/it]Validation Accuracy: 0.8669258978249874
# Train loss: 0.28706888220848953
# Epoch:  60%|██████    | 3/5 [34:56<23:17, 698.86s/it]Validation Accuracy: 0.8666350531107738
# Train loss: 0.26255775001885256
# Epoch:  80%|████████  | 4/5 [46:35<11:38, 698.70s/it]Validation Accuracy: 0.8645612038442083
# Train loss: 0.23857382561691684
# Epoch: 100%|██████████| 5/5 [58:14<00:00, 698.85s/it]Validation Accuracy: 0.8637076378351036

# Weighted F1 Score: 0.8647974263572064
# Macro F1 Score: 0.7813540293511745
# Accuracy score: 0.8658351512852129 

# ====================================================================================================

# Model Summary:
# Language: English
# Seed value: 2019
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.36447684346511877
# Epoch:  20%|██        | 1/5 [11:38<46:34, 698.69s/it]Validation Accuracy: 0.8676909458775922
# Train loss: 0.31646106722277106
# Epoch:  40%|████      | 2/5 [23:17<34:56, 698.75s/it]Validation Accuracy: 0.8686203844208397
# Train loss: 0.2855927807737425
# Epoch:  60%|██████    | 3/5 [34:56<23:17, 698.85s/it]Validation Accuracy: 0.8704350025290845
# Train loss: 0.26328138579042504
# Epoch:  80%|████████  | 4/5 [46:35<11:38, 698.83s/it]Validation Accuracy: 0.8663568538189176
# Train loss: 0.2400031902544877
# Epoch: 100%|██████████| 5/5 [58:14<00:00, 698.97s/it]Validation Accuracy: 0.8675961052099139

# Weighted F1 Score: 0.8588953965463227
# Macro F1 Score: 0.7707407004993247
# Accuracy score: 0.8606658446362515 

# ====================================================================================================

# Model Summary:
# Language: English
# Seed value: 2020
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.3671970392918344
# Epoch:  20%|██        | 1/5 [11:38<46:34, 698.68s/it]Validation Accuracy: 0.862373545776429
# Train loss: 0.31486930572499483
# Epoch:  40%|████      | 2/5 [23:17<34:55, 698.58s/it]Validation Accuracy: 0.8627529084471421
# Train loss: 0.2879878534906544
# Epoch:  60%|██████    | 3/5 [34:55<23:17, 698.53s/it]Validation Accuracy: 0.869663631765301
# Train loss: 0.26159273587557313
# Epoch:  80%|████████  | 4/5 [46:33<11:38, 698.54s/it]Validation Accuracy: 0.8698659585230146
# Train loss: 0.23927275580651824
# Epoch: 100%|██████████| 5/5 [58:13<00:00, 698.61s/it]Validation Accuracy: 0.8680703085483055

# Weighted F1 Score: 0.8627163383793262
# Macro F1 Score: 0.7756112313720112
# Accuracy score: 0.8653134781371526 

# ====================================================================================================

# Model Summary:
# Language: English
# Seed value: 2021
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.3658158818941015
# Epoch:  20%|██        | 1/5 [11:39<46:36, 699.20s/it]Validation Accuracy: 0.864270359129995
# Train loss: 0.3129915121487293
# Epoch:  40%|████      | 2/5 [23:16<34:56, 698.74s/it]Validation Accuracy: 0.8564871016691957
# Train loss: 0.28509787482417653
# Epoch:  60%|██████    | 3/5 [34:54<23:16, 698.48s/it]Validation Accuracy: 0.8627529084471421
# Train loss: 0.25811701204321685
# Epoch:  80%|████████  | 4/5 [46:33<11:38, 698.53s/it]Validation Accuracy: 0.863321952453212
# Train loss: 0.2355335181511966
# Epoch: 100%|██████████| 5/5 [58:12<00:00, 698.58s/it]Validation Accuracy: 0.8639858371269601

# Weighted F1 Score: 0.858353043827432
# Macro F1 Score: 0.7701313790851088
# Accuracy score: 0.8599544721616238 

# ====================================================================================================

# Model Summary:
# Language: English
# Seed value: 2022
# Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
# - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
# - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
# Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
# You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
# Epoch:   0%|          | 0/5 [00:00<?, ?it/s]Train loss: 0.36561783764542843
# Epoch:  20%|██        | 1/5 [11:38<46:35, 698.86s/it]Validation Accuracy: 0.8469081942336874
# Train loss: 0.31494393829137585
# Epoch:  40%|████      | 2/5 [23:16<34:55, 698.48s/it]Validation Accuracy: 0.8618045017703592
# Train loss: 0.2888010124965239
# Epoch:  60%|██████    | 3/5 [34:54<23:16, 698.38s/it]Validation Accuracy: 0.8631133029843197
# Train loss: 0.2635283211095522
# Epoch:  80%|████████  | 4/5 [46:33<11:38, 698.62s/it]Validation Accuracy: 0.8645422357106728
# Train loss: 0.24019365908233611
# Epoch: 100%|██████████| 5/5 [58:13<00:00, 698.70s/it]Validation Accuracy: 0.8651239251390996

# Weighted F1 Score: 0.8609575842541556
# Macro F1 Score: 0.7716388924058272
# Accuracy score: 0.8643175566726738 

# ====================================================================================================
# The Average  Weighted F1-Score of the Language  English is: 0.8611439578728886
# The Average  Macro F1-Score of the Language  English is: 0.7738952465426893
# ========================================================================================================================================================================================================
# Average Weighted F1-Score0.5617914925533054and Average Macro F1-Score0.4359955364259525of the languageFrench 

# Average Weighted F1-Score0.8988776449510304and Average Macro F1-Score0.8471303948622133of the languageArabic 

# Average Weighted F1-Score0.8611439578728886and Average Macro F1-Score0.7738952465426893of the languageEnglish 

# ====================================================================================================English====================================================================================================